# Let's Help Santa!

## 1. Download the Data
"Uber Movement" data -- Travel Times by Month (All Days)

## 2. Build the Uber Graph

Read the dataset at hand, and build a graph in which nodes correspond to locations, and
undirected weighted edges correspond to the mean traveling times between each pair of locations (only December). Add the mean of the coordinates of each region's polygon corners (a 2-D vector) as an attribute to the corresponding vertex.
The graph will contain some isolated nodes (extra nodes existing in the Geo Boundaries JSON file) and a few small connected components. Remove such nodes and just keep the largest connected component of the graph. In addition, merge duplicate edges by averaging their weights. We will refer to this cleaned graph as G afterwards.

In [7]:
import igraph as ig
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from statistics import mean

In [2]:
!pwd

/Users/zxiao/Documents/ECE232E_largescalenetwork/Large_Scale_Social_Complex_Network/Project4


In [3]:
directory = os.path.abspath('data/')
uber_data = pd.read_csv(directory + '/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv')

In [4]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144062 entries, 0 to 5144061
Data columns (total 7 columns):
sourceid                                    int64
dstid                                       int64
month                                       int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtypes: float64(4), int64(3)
memory usage: 274.7 MB


In [5]:
uber_data.head()

,sourceid,dstid,month,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,17,296,10,1109.36,492.50,1021.90,1.48
1,28,186,10,1625.16,475.09,1565.73,1.30
2,758,972,12,953.55,269.46,916.93,1.33
3,1212,547,10,2053.39,648.63,1953.97,1.37
4,1299,1221,11,1467.54,539.64,1370.82,1.45


In [6]:
uber_condensed_data = uber_data.drop(['standard_deviation_travel_time','geometric_mean_travel_time','geometric_standard_deviation_travel_time'], axis = 1)
# Keep December data
df = uber_condensed_data[uber_condensed_data.month==12].reset_index(drop=True)
df = df.drop(['month'], axis=1)
df.head()

,sourceid,dstid,mean_travel_time
0,758,972,953.55
1,1197,2006,1213.18
2,2652,620,828.92
3,2427,2693,1742.80
4,2653,610,699.03


In [28]:
# Merge duplicate edges
data = np.asarray(df)
# create dictionary, key as tuple of locations, value as a list of mean_travel_time(s)
paths = {} 

# Construct dictionary, append mean_travel_time to the same location pair
for row in data:
    locations = tuple(np.sort(row[0:2]))
    if locations in paths:
        paths[locations].append(row[2])
    else:
        paths[locations] = [row[2]]

# Open function to open the file "uber_december.txt" and write to it
with open("data/uber_dec.txt","w") as f:
    for locations in paths:
        paths[locations] = mean(paths[locations])
        merged_rows = "{} {} {:.2f} \n".format(int(locations[0]), int(locations[1]), paths[locations])
        f.write(merged_rows)

In [105]:
# Load the Geo Boundries data

with open('data/los_angeles_censustracts.json','r') as file:
    geo_bounds = json.load(file)
geo_bounds = geo_bounds['features']

geo_bounds

[{'geometry': {'coordinates': [[[-118.11683, 34.107225],
     [-118.116311, 34.10648],
     [-118.115782, 34.106729],
     [-118.115717, 34.106606],
     [-118.115696, 34.106563],
     [-118.115653, 34.106468],
     [-118.115613, 34.106402],
     [-118.115488, 34.10611],
     [-118.115409, 34.10611],
     [-118.11532, 34.106112],
     [-118.115171, 34.106113],
     [-118.115174, 34.106068],
     [-118.115174, 34.105996],
     [-118.11517, 34.105924],
     [-118.115157, 34.105829],
     [-118.115142, 34.105758],
     [-118.115125, 34.105696],
     [-118.115762, 34.105696],
     [-118.115543, 34.105381],
     [-118.113094, 34.101877],
     [-118.11306, 34.10185],
     [-118.112989, 34.101707],
     [-118.114592, 34.100991],
     [-118.114876, 34.100861],
     [-118.116693, 34.099987],
     [-118.117786, 34.099464],
     [-118.120994, 34.097937],
     [-118.12254, 34.097193],
     [-118.124152, 34.096419],
     [-118.125194, 34.095935],
     [-118.126868, 34.09516],
     [-118.127179, 34.

In [138]:
print('\n Polygon type:', (geo_bounds[0]['geometry']['type']))
print('\n Number of Polygons:', len(geo_bounds[0]['geometry']['coordinates']))
print('Polygon 1', geo_bounds[0]['geometry']['coordinates'][0])


 Polygon type: Polygon

 Number of Polygons: 1
Polygon 1 [[-118.11683, 34.107225], [-118.116311, 34.10648], [-118.115782, 34.106729], [-118.115717, 34.106606], [-118.115696, 34.106563], [-118.115653, 34.106468], [-118.115613, 34.106402], [-118.115488, 34.10611], [-118.115409, 34.10611], [-118.11532, 34.106112], [-118.115171, 34.106113], [-118.115174, 34.106068], [-118.115174, 34.105996], [-118.11517, 34.105924], [-118.115157, 34.105829], [-118.115142, 34.105758], [-118.115125, 34.105696], [-118.115762, 34.105696], [-118.115543, 34.105381], [-118.113094, 34.101877], [-118.11306, 34.10185], [-118.112989, 34.101707], [-118.114592, 34.100991], [-118.114876, 34.100861], [-118.116693, 34.099987], [-118.117786, 34.099464], [-118.120994, 34.097937], [-118.12254, 34.097193], [-118.124152, 34.096419], [-118.125194, 34.095935], [-118.126868, 34.09516], [-118.127179, 34.095604], [-118.127898, 34.096637], [-118.12827, 34.097221], [-118.128556, 34.097623], [-118.128613, 34.097689], [-118.129597, 34

In [135]:
# MultiPolygon cases
print('\n Polygon type:', (geo_bounds[53]['geometry']['type']))
print('\n Number of Polygons:', len(geo_bounds[53]['geometry']['coordinates']))
print('Polygon 1', geo_bounds[53]['geometry']['coordinates'][0][0])
print('\n Polygon 2', geo_bounds[53]['geometry']['coordinates'][1][0])


 Polygon type: MultiPolygon

 Number of Polygons: 2
Polygon 1 [[-117.922484, 34.114307], [-117.921951, 34.114311], [-117.92077, 34.114316], [-117.920785, 34.11421], [-117.920812, 34.113002], [-117.920812, 34.11287], [-117.920902, 34.11173], [-117.920964, 34.110598], [-117.923242, 34.110592], [-117.924231, 34.110582], [-117.924226, 34.110507], [-117.924214, 34.109433], [-117.92424, 34.109433], [-117.924235, 34.108802], [-117.920933, 34.108834], [-117.920954, 34.108117], [-117.920958, 34.108057], [-117.920957, 34.108035], [-117.920973, 34.10749], [-117.920972, 34.107436], [-117.920982, 34.107159], [-117.920985, 34.10701], [-117.922126, 34.106999], [-117.924215, 34.106984], [-117.924863, 34.106965], [-117.925363, 34.106965], [-117.925355, 34.107466], [-117.925362, 34.107556], [-117.925347, 34.108149], [-117.925338, 34.108686], [-117.925341, 34.10885], [-117.925322, 34.110001], [-117.9253, 34.110628], [-117.927484, 34.110626], [-117.927529, 34.108799], [-117.928097, 34.108807], [-117.9284

In [140]:
"""
Create a dictionary 
-- key: movement_ID of the geometry
-- value: display name, average coordinate of the geometry
"""
loc_dict = {}

for i, item in enumerate(geo_bounds):
    movement_ID = item['properties']['MOVEMENT_ID']
    
    display_name = item['properties']['DISPLAY_NAME']
    # record the coordinates and take average in the geometry associated with each movement_ID
    if item['geometry']['type'] == 'Polygon':
        coordinates = item['geometry']['coordinates'][0]
        
    elif item['geometry']['type']== 'MultiPolygon':
        num_polygons = len(item['geometry']['coordinates'])
        print('Number of polygons in multipolygon:', num_polygons)
        coordinates = item['geometry']['coordinates'][0][0]
        #print(coordinates1)
        #coordinates2 = item['geometry']['coordinates'][1]
        #print(coordinates2)
        for j in range(1, num_polygons-1):           
            coordinates = np.vstack((coordinates, item['geometry']['coordinates'][j][0]))

    mean_coordinate = np.mean(np.asarray(coordinates), axis = 0)
    loc_dict[movement_ID] = (display_name, mean_coordinate)
    print(movement_ID, mean_coordinate,'\n')
    

1 [-118.12053321   34.10309557] 

2 [-118.13785063   34.09645121] 

3 [-118.13138209   34.09626386] 

4 [-118.13224544   34.10349303] 

5 [-118.14492317   34.0986815 ] 

6 [-118.1528085   34.098628 ] 

7 [-118.15075124   34.08341963] 

8 [-118.15266639   34.09029573] 

9 [-118.15023891   34.09595766] 

10 [-118.14184446   34.08538654] 

11 [-118.14162277   34.08013327] 

12 [-118.13948648   34.07262509] 

13 [-118.12911933   34.08759475] 

14 [-118.11656383   34.09585388] 

15 [-118.11315003   34.08121859] 

16 [-118.12013004   34.08627508] 

17 [-118.12602952   34.08456533] 

18 [-118.12195981   34.07206641] 

19 [-118.12637009   34.07659087] 

20 [-118.14237141   34.06371094] 

21 [-118.1589895    34.06994802] 

22 [-118.14907131   34.06458611] 

23 [-118.02458802   34.16641559] 

24 [-118.05076541   34.15513302] 

25 [-118.05385033   34.14265366] 

26 [-118.04590074   34.14288099] 

27 [-118.05794788   34.13033512] 

28 [-118.06251639   34.12621528] 

29 [-118.02466274   34.13765353

441 [-118.2362856    33.97771228] 

442 [-118.23195906   33.9775516 ] 

443 [-118.22426074   33.97671916] 

444 [-118.21638926   33.98254978] 

445 [-118.21664846   33.9790122 ] 

446 [-118.2154226    33.97436889] 

447 [-118.20808716   33.98220595] 

448 [-118.20871327   33.97794507] 

449 [-118.20700069   33.97465208] 

450 [-118.20089513   33.98628645] 

451 [-118.19451017   33.98522552] 

452 [-118.190518     33.98513294] 

453 [-118.20724587   33.96758759] 

454 [-118.1999428    33.96679709] 

Number of polygons in multipolygon: 3
455 [-117.85763072   34.00554197] 

456 [-117.83443241   34.01461976] 

457 [-117.9856075    34.06277838] 

458 [-117.98551295   34.06180248] 

459 [-117.99472595   34.05551724] 

460 [-117.940085    34.0344212] 

461 [-117.93217161   34.0267175 ] 

462 [-117.92185528   34.01498936] 

463 [-117.90876114   34.00664343] 

464 [-117.96422843   34.02042817] 

465 [-117.91463932   34.00165302] 

Number of polygons in multipolygon: 3
466 [-117.87811432   33.99


767 [-118.44315005   34.27522591] 

768 [-118.44799424   34.26159714] 

769 [-118.46481275   34.253502  ] 

770 [-118.46253665   34.26272541] 

771 [-118.48438148   34.26541926] 

772 [-118.48706248   34.25354789] 

773 [-118.4912426   34.2677474] 

774 [-118.51223755   34.27641308] 

775 [-118.51997097   34.26691994] 

776 [-118.54918363   34.26508911] 

777 [-118.53919618   34.25337168] 

778 [-118.53450685   34.26974513] 

779 [-118.5220121    34.26204912] 

780 [-118.51596593   34.25508227] 

781 [-118.49281834   34.25666208] 

782 [-118.58981447   34.26815043] 

783 [-118.56354069   34.26758733] 

784 [-118.6102306    34.22807996] 

785 [-118.6036088    34.22858093] 

786 [-118.59202711   34.22658108] 

787 [-118.61703425   34.24546598] 

788 [-118.56085991   34.24479202] 

789 [-118.57437451   34.24140674] 

790 [-118.593677     34.25369413] 

791 [-118.56464114   34.25320243] 

792 [-118.56338305   34.23100405] 

793 [-118.58762217   34.23532122] 

794 [-118.58693752   34.22394

1100 [-118.3509417   34.1009097] 

1101 [-118.34137107   34.10069007] 

1102 [-118.33064041   34.10416976] 

1103 [-118.33499296   34.10399402] 

1104 [-118.31985212   34.10396042] 

1105 [-118.30292918   34.10349818] 

1106 [-118.30665533   34.10415867] 

1107 [-118.3109394   34.0995622] 

1108 [-118.303324    34.0998065] 

1109 [-118.3327684   34.0981628] 

1110 [-118.32373133   34.09377133] 

1111 [-118.32881967   34.09345483] 

1112 [-118.31200371   34.09329529] 

1113 [-118.31854429   34.09338857] 

1114 [-118.3211149    34.10331545] 

1115 [-118.30533567   34.09571822] 

1116 [-118.30410033   34.093657  ] 

1117 [-118.2965617   34.0979014] 

1118 [-118.29360657   34.09298929] 

1119 [-118.297561     34.09318533] 

1120 [-118.28368617   34.09210775] 

1121 [-118.28597123   34.09562577] 

1122 [-118.28695655   34.08998109] 

1123 [-118.287405     34.08450614] 

1124 [-118.2960189   34.0857176] 

1125 [-118.30428941   34.08702594] 

1126 [-118.30356527   34.08758493] 

1127 [-118.31

1433 [-118.27806262   33.99309738] 

1434 [-118.32179652   34.01022162] 

1435 [-118.33073046   34.01812379] 

1436 [-118.33036094   34.0064268 ] 

1437 [-118.32465042   33.997746  ] 

1438 [-118.32870046   34.00236896] 

1439 [-118.32914212   33.99221773] 

1440 [-118.3295256   33.9881266] 

1441 [-118.32101038   33.98000863] 

1442 [-118.32793871   33.980538  ] 

1443 [-118.33457446   33.98137815] 

1444 [-118.35057805   33.98423462] 

1445 [-118.3216741    33.97381945] 

1446 [-118.33131062   33.97243367] 

1447 [-118.36805117   34.01405508] 

1448 [-118.34113841   34.01062715] 

1449 [-118.35142253   34.01611595] 

1450 [-118.34704889   34.01739378] 

1451 [-118.34925273   34.01456519] 

1452 [-118.35108416   34.00533058] 

1453 [-118.29028868   33.98355324] 

1454 [-118.28337741   33.98441659] 

1455 [-118.29827233   33.98546667] 

1456 [-118.30951333   33.98506419] 

1457 [-118.3092458   33.9801986] 

1458 [-118.29276733   33.98001833] 

1459 [-118.3001535   33.9774654] 

1460 [-

1764 [-118.12233921   34.05827176] 

1765 [-118.11244847   34.05840403] 

1766 [-118.11390298   34.04849873] 

1767 [-118.13811197   34.04037746] 

1768 [-118.13388917   34.04181381] 

1769 [-118.11516446   34.03631878] 

1770 [-118.15242317   34.03803105] 

1771 [-118.08896191   33.93049491] 

1772 [-118.06790877   33.92320013] 

1773 [-118.07441618   33.92436602] 

1774 [-118.08360569   33.92757271] 

1775 [-118.09271555   33.92162592] 

1776 [-118.09748372   33.92462468] 

1777 [-118.1049893    33.90943009] 

1778 [-118.09471497   33.91164836] 

1779 [-118.08626927   33.911992  ] 

1780 [-118.08660765   33.90637371] 

1781 [-118.07387104   33.90907895] 

1782 [-118.06318653   33.91264343] 

1783 [-118.0586229    33.90646433] 

1784 [-118.05090012   33.89913495] 

1785 [-118.0762795    33.89162508] 

1786 [-118.06404115   33.89134243] 

1787 [-118.07337203   33.89879415] 

1788 [-118.09094483   33.89772771] 

1789 [-118.0917493    33.89242295] 

1790 [-118.1038304    33.89306996] 

1

2087 [-117.93194464   34.05343989] 

2088 [-117.94547543   34.0552299 ] 

2089 [-117.93646314   34.06729147] 

2090 [-117.95440308   34.06620809] 

2091 [-117.96254518   34.06370241] 

2092 [-117.96062532   34.06086145] 

2093 [-117.95634804   34.05139915] 

2094 [-117.91554883   34.03146192] 

2095 [-117.92323597   34.04004566] 

2096 [-117.91583064   34.04390227] 

2097 [-117.91576905   34.03902368] 

2098 [-117.89365873   34.0510314 ] 

2099 [-117.90174038   34.03251129] 

2100 [-117.90839889   34.02365067] 

2101 [-117.89818917   34.02209941] 

2102 [-117.89783202   34.01728623] 

2103 [-117.8751469    34.00798446] 

Number of polygons in multipolygon: 2
2104 [-117.88361993   34.00478544] 

Number of polygons in multipolygon: 3
2105 [-117.88970683   34.00723112] 

2106 [-118.35196152   34.09139217] 

2107 [-118.35179363   34.08993829] 

2108 [-118.36578606   34.09247796] 

2109 [-118.37279253   34.09243982] 

2110 [-118.38068395   34.08116623] 

2111 [-118.38792886   34.08807885] 


2415 [-118.29938545   33.94303345] 

2416 [-118.30351386   33.93483832] 

2417 [-118.29872907   33.93502379] 

2418 [-118.29441243   33.9353652 ] 

2419 [-118.31418855   33.93518991] 

2420 [-118.37018188   33.96814512] 

Number of polygons in multipolygon: 2
2421 [-118.36354777   33.9436182 ] 

2422 [-118.36609083   33.94044815] 

2423 [-118.36347711   33.93421103] 

2424 [-118.35049242   33.93439408] 

Number of polygons in multipolygon: 2
2425 [-118.34776711   33.94231056] 

2426 [-118.34882577   33.93999633] 

2427 [-118.37105149   33.9230834 ] 

2428 [-118.36484003   33.90852444] 

2429 [-118.33146552   33.91127652] 

2430 [-118.31280837   33.92433653] 

2431 [-118.29991938   33.92666387] 

2432 [-118.30105862   33.92056203] 

2433 [-118.33654964   33.88733836] 

2434 [-118.32957539   33.89751671] 

2435 [-118.3028484    33.74075612] 

2436 [-118.42332207   33.90297007] 

2437 [-118.41693521   33.89136561] 

2438 [-118.41094278   33.87995286] 

2439 [-118.40584589   33.86988847] 


In [148]:
loc_dict

{'2549': ('Census Tract 110606', array([-118.0056688,   33.8770831])),
 '212': ('Census Tract 542700', array([-118.23738959,   33.90151074])),
 '2381': ('Census Tract 541001', array([-118.27227764,   33.8936948 ])),
 '26': ('Census Tract 430721', array([-118.04590074,   34.14288099])),
 '1823': ('Census Tract 461902', array([-118.14948823,   34.14911746])),
 '1639': ('Census Tract 294110', array([-118.26024189,   33.79904073])),
 '1245': ('Census Tract 208720', array([-118.2805606 ,   34.06364133])),
 '80': ('Census Tract 533603', array([-118.19665772,   33.97217828])),
 '628': ('Census Tract 573201', array([-118.18455684,   33.80179952])),
 '1825': ('Census Tract 462002', array([-118.14855507,   34.16147414])),
 '1729': ('Census Tract 430301', array([-117.99399975,   34.16600654])),
 '2457': ('Census Tract 702102', array([-118.48561034,   33.99744194])),
 '1707': ('Census Tract 540502', array([-118.21348578,   33.91763146])),
 '1450': ('Census Tract 236203', array([-118.34704889,   34

In [156]:
loc_dict['1377'][0]

'Census Tract 222500'

The graph will contain some isolated nodes (extra nodes existing in the Geo Boundaries JSONfile)  and  a  few  small  connected  components.   Remove  such  nodes  and  just  keep  the  largestconnected  component  of  the  graph. In  addition,  merge  duplicate  edges  by  averaging  their weights. We will refer to this cleaned graph as G afterwards.

### Question 6: 
Report the number of nodes and edges in G.

In [142]:
uber_graph = ig.Graph.Read_Ncol('data/uber_dec.txt', directed = False)
ig.summary(uber_graph)

gcc = uber_graph.components().giant()
ig.summary(gcc)

# Write the graph to file
ig.Graph.write_ncol(gcc, 'data/uber_gcc.txt')

IGRAPH UNW- 2649 1004955 -- 
+ attr: name (v), weight (e)
IGRAPH UNW- 2649 1004955 -- 
+ attr: name (v), weight (e)


### Question 7:
Build a minimum spanning tree (MST) of graph G. Report the street addresses (census tract) of the two endpoints of a few edges. Are the results intuitive?

In [293]:
# reference: https://igraph.org/python/doc/igraph-pysrc.html
mst = gcc.spanning_tree(weights = gcc.es["weight"])
vertices = mst.vs()
ig.summary(mst)

for i, e in enumerate(mst.es()):# igraph.EdgeSeq object
    st = e.tuple
    # print(e, e['weight'], st)
    start = vertices[st[0]]['name']
    end = vertices[st[1]]['name']
    print(loc_dict[start][0],'--',loc_dict[end][0], ', weight:', e["weight"])

IGRAPH UNW- 2649 2648 -- 
+ attr: name (v), weight (e)
Census Tract 139701 -- Census Tract 980024 , weight: 156.1
Census Tract 228320 -- Census Tract 228600 , weight: 92.86
Census Tract 001705 -- Census Tract 001601 , weight: 31.82
Census Tract 264000 -- Census Tract 262303 , weight: 167.78
Census Tract 086902 -- Census Tract 087802 , weight: 61.73
Census Tract 602104 -- Census Tract 602002 , weight: 30.58
Census Tract 224410 -- Census Tract 221710 , weight: 86.34
Census Tract 273300 -- Census Tract 273402 , weight: 118.91
Census Tract 703002 -- Census Tract 703002 , weight: 103.02
Census Tract 602506 -- Census Tract 602507 , weight: 82.41
Census Tract 232120 -- Census Tract 232110 , weight: 96.88
Census Tract 431800 -- Census Tract 431800 , weight: 55.91
Census Tract 191000 -- Census Tract 190301 , weight: 124.41
Census Tract 189201 -- Census Tract 189101 , weight: 109.71
Census Tract 011101 -- Census Tract 011102 , weight: 41.06
Census Tract 400501 -- Census Tract 400800 , weight: 66

Census Tract 534203 -- Census Tract 550800 , weight: 152.48
Census Tract 553601 -- Census Tract 553602 , weight: 49.81
Census Tract 195902 -- Census Tract 195901 , weight: 77.78
Census Tract 550700 -- Census Tract 550601 , weight: 83.86
Census Tract 271200 -- Census Tract 267600 , weight: 157.41
Census Tract 109700 -- Census Tract 109300 , weight: 65.81
Census Tract 192002 -- Census Tract 194401 , weight: 120.53
Census Tract 573403 -- Census Tract 573403 , weight: 61.79
Census Tract 570204 -- Census Tract 554302 , weight: 114.03
Census Tract 088104 -- Census Tract 088105 , weight: 70.67
Census Tract 530003 -- Census Tract 530003 , weight: 97.9
Census Tract 269500 -- Census Tract 269602 , weight: 91.66
Census Tract 001504 -- Census Tract 001507 , weight: 64.09
Census Tract 404702 -- Census Tract 404701 , weight: 99.92
Census Tract 139001 -- Census Tract 139600 , weight: 113.75
Census Tract 460301 -- Census Tract 460302 , weight: 99.03
Census Tract 531800 -- Census Tract 531800 , weight:

Census Tract 208501 -- Census Tract 208502 , weight: 77.06
Census Tract 406402 -- Census Tract 408005 , weight: 122.17
Census Tract 701801 -- Census Tract 702300 , weight: 83.41
Census Tract 217002 -- Census Tract 216700 , weight: 82.47
Census Tract 533902 -- Census Tract 533901 , weight: 110.18
Census Tract 269100 -- Census Tract 217001 , weight: 124.38
Census Tract 542104 -- Census Tract 542103 , weight: 56.2
Census Tract 408212 -- Census Tract 408212 , weight: 70.78
Census Tract 189702 -- Census Tract 189701 , weight: 167.42
Census Tract 239310 -- Census Tract 239320 , weight: 85.21
Census Tract 106405 -- Census Tract 106010 , weight: 145.96
Census Tract 407802 -- Census Tract 407802 , weight: 110.27
Census Tract 276000 -- Census Tract 277000 , weight: 138.41
Census Tract 541100 -- Census Tract 543000 , weight: 86.14
Census Tract 267901 -- Census Tract 267100 , weight: 150.78
Census Tract 550601 -- Census Tract 550602 , weight: 134.2
Census Tract 320202 -- Census Tract 320100 , weig

Census Tract 265201 -- Census Tract 265602 , weight: 65.47
Census Tract 110007 -- Census Tract 110007 , weight: 25.42
Census Tract 310602 -- Census Tract 310601 , weight: 138.18
Census Tract 215101 -- Census Tract 214503 , weight: 87.89
Census Tract 570902 -- Census Tract 570901 , weight: 123.43
Census Tract 407301 -- Census Tract 406902 , weight: 63.67
Census Tract 700102 -- Census Tract 192001 , weight: 114.4
Census Tract 192300 -- Census Tract 191820 , weight: 88.43
Census Tract 431502 -- Census Tract 430803 , weight: 112.75
Census Tract 267402 -- Census Tract 267502 , weight: 93.67
Census Tract 272201 -- Census Tract 272302 , weight: 96.38
Census Tract 531504 -- Census Tract 531604 , weight: 98.47
Census Tract 620800 -- Census Tract 621001 , weight: 162.49
Census Tract 011000 -- Census Tract 001901 , weight: 109.5
Census Tract 570404 -- Census Tract 542200 , weight: 55.72
Census Tract 433801 -- Census Tract 433902 , weight: 76.01
Census Tract 554515 -- Census Tract 554900 , weight:

Census Tract 651222 -- Census Tract 651221 , weight: 155.38
Census Tract 115301 -- Census Tract 113301 , weight: 135.79
Census Tract 407001 -- Census Tract 406901 , weight: 97.65
Census Tract 463601 -- Census Tract 463500 , weight: 144.19
Census Tract 405500 -- Census Tract 405301 , weight: 83.43
Census Tract 543201 -- Census Tract 542501 , weight: 124.12
Census Tract 502200 -- Census Tract 502301 , weight: 90.9
Census Tract 135201 -- Census Tract 135202 , weight: 127.12
Census Tract 576403 -- Census Tract 576402 , weight: 61.11
Census Tract 134001 -- Census Tract 134002 , weight: 64.62
Census Tract 117407 -- Census Tract 117408 , weight: 81.17
Census Tract 087802 -- Census Tract 087803 , weight: 69.67
Census Tract 403901 -- Census Tract 404202 , weight: 64.13
Census Tract 701201 -- Census Tract 701202 , weight: 102.39
Census Tract 302103 -- Census Tract 302004 , weight: 116.82
Census Tract 195803 -- Census Tract 195804 , weight: 77.51
Census Tract 501001 -- Census Tract 501002 , weigh

Census Tract 980010 -- Census Tract 206010 , weight: 67.86
Census Tract 500800 -- Census Tract 502200 , weight: 102.5
Census Tract 432300 -- Census Tract 432101 , weight: 98.38
Census Tract 088002 -- Census Tract 088001 , weight: 63.92
Census Tract 212800 -- Census Tract 218110 , weight: 97.82
Census Tract 700101 -- Census Tract 189902 , weight: 87.95
Census Tract 576901 -- Census Tract 576403 , weight: 73.82
Census Tract 480302 -- Census Tract 481002 , weight: 118.34
Census Tract 240020 -- Census Tract 240010 , weight: 77.44
Census Tract 406411 -- Census Tract 406412 , weight: 104.8
Census Tract 571101 -- Census Tract 571000 , weight: 128.16
Census Tract 183401 -- Census Tract 183300 , weight: 102.44
Census Tract 087401 -- Census Tract 087200 , weight: 98.89
Census Tract 302506 -- Census Tract 302505 , weight: 90.03
Census Tract 571300 -- Census Tract 571400 , weight: 140.97
Census Tract 572201 -- Census Tract 572202 , weight: 138.66
Census Tract 293307 -- Census Tract 293306 , weight

Census Tract 601402 -- Census Tract 601401 , weight: 69.5
Census Tract 123520 -- Census Tract 123602 , weight: 98.06
Census Tract 235201 -- Census Tract 235202 , weight: 117.14
Census Tract 087902 -- Census Tract 088104 , weight: 70.08
Census Tract 267902 -- Census Tract 267100 , weight: 130.57
Census Tract 206300 -- Census Tract 206200 , weight: 128.31
Census Tract 577602 -- Census Tract 577602 , weight: 113.03
Census Tract 536200 -- Census Tract 551600 , weight: 111.7
Census Tract 800203 -- Census Tract 800204 , weight: 110.78
Census Tract 650001 -- Census Tract 650004 , weight: 111.25
Census Tract 670405 -- Census Tract 670407 , weight: 114.03
Census Tract 574800 -- Census Tract 574700 , weight: 147.16
Census Tract 234700 -- Census Tract 234600 , weight: 106.9
Census Tract 482201 -- Census Tract 482102 , weight: 134.19
Census Tract 482521 -- Census Tract 482522 , weight: 81.48
Census Tract 621400 -- Census Tract 621326 , weight: 128.62
Census Tract 541801 -- Census Tract 541801 , we

Census Tract 110108 -- Census Tract 110106 , weight: 86.4
Census Tract 433700 -- Census Tract 433700 , weight: 96.17
Census Tract 099801 -- Census Tract 088106 , weight: 25.08
Census Tract 203900 -- Census Tract 531101 , weight: 46.52
Census Tract 570001 -- Census Tract 570901 , weight: 130.51
Census Tract 481500 -- Census Tract 482301 , weight: 74.36
Census Tract 980002 -- Census Tract 543903 , weight: 97.88
Census Tract 434003 -- Census Tract 433902 , weight: 115.4
Census Tract 224410 -- Census Tract 224200 , weight: 50.76
Census Tract 121600 -- Census Tract 121020 , weight: 79.84
Census Tract 550202 -- Census Tract 552002 , weight: 139.75
Census Tract 701302 -- Census Tract 701501 , weight: 112.85
Census Tract 297602 -- Census Tract 297201 , weight: 106.77
Census Tract 503601 -- Census Tract 503106 , weight: 141.21
Census Tract 190201 -- Census Tract 189600 , weight: 115.91
Census Tract 502301 -- Census Tract 502700 , weight: 107.41
Census Tract 204600 -- Census Tract 206050 , weigh

Census Tract 217002 -- Census Tract 269601 , weight: 115.12
Census Tract 533503 -- Census Tract 533502 , weight: 70.41
Census Tract 110605 -- Census Tract 110604 , weight: 88.99
Census Tract 572301 -- Census Tract 980014 , weight: 21.12
Census Tract 183820 -- Census Tract 185100 , weight: 62.2
Census Tract 086602 -- Census Tract 086601 , weight: 96.8
Census Tract 408133 -- Census Tract 408138 , weight: 73.58
Census Tract 213402 -- Census Tract 213401 , weight: 81.28
Census Tract 183101 -- Census Tract 183104 , weight: 114.09
Census Tract 186401 -- Census Tract 186301 , weight: 61.86
Census Tract 603704 -- Census Tract 603500 , weight: 107.38
Census Tract 532700 -- Census Tract 533002 , weight: 122.97
Census Tract 576001 -- Census Tract 576001 , weight: 131.31
Census Tract 276601 -- Census Tract 276604 , weight: 136.75
Census Tract 552602 -- Census Tract 552700 , weight: 122.72
Census Tract 570003 -- Census Tract 570002 , weight: 118.68
Census Tract 265202 -- Census Tract 265510 , weigh

In [287]:
print(mst.es())

### Question 8:
Determine what percentage of triangles in the graph (sets of 3 points on the map) satisfy the triangle inequality. 
You do not need to inspect all triangles, you can just estimate by random sampling of 1000 triangles.

In [260]:
def satisfy_triangular_inequality(indices_set):
    weights = []
    
    for edge in indices_set:
        weights.append(gcc_edges[edge]["weight"])
    
    if weights[0] + weights[1] > weights[2] \
        and weights[2] + weights[1] > weights[0] \
        and weights[0] + weights[2] > weights[1]:
        return weights, True
    else: 
        return weights, False

In [254]:
# loc_dict[gcc.vs[gcc_edges[6].source]['name']][0]

gcc.vs[gcc_edges[6].source]['name']

'309'

In [297]:
num_samples = 1000
max_index = len(gcc.vs()) - 1 #2648
gcc_vertices = gcc.vs()
gcc_edges = gcc.es()


all_sets = set() # contain a set of evaluated indices_set
num_satisfied = 0

i = 0
while i < num_samples:
    
    indices=[] # store the 3 random generated indices
    indices_set = set() # three indices in tuple, sorted in tuple
    
    for j in range(3):
        # Generate three random indices
        indices.append(random.randint(0, max_index))
    indices_set = tuple(sorted(indices))
    # print(indices_set)
    
    if not indices_set in all_sets:
        all_sets.add(indices_set) 
        weights, satisfy = satisfy_triangular_inequality(indices_set)
    
        if satisfy:
            num_satisfied += 1
    else:
        i += 1
        break
    
    print('-'*20,'triangle',i+1,'-'*20)
    
    

    edges = []
    start = []
    end = []
    
    index = 0 # keep track of the index of the edge in the indices_set
    for edge in indices_set: 
        # append display name of the vertices
        source_vertex = gcc.vs[gcc_edges[edge].source]['name']
        target_vertex = gcc.vs[gcc_edges[edge].target]['name']

        start.append(loc_dict[source_vertex][0])
        end.append(loc_dict[target_vertex][0])
        index += 1
        
        print('edge ' + str(index) + ':', start[index-1], '--', end[index-1], '\nweight:', weights[index-1])
    
    if satisfy:
        print('>>> Satisfy Triangle Inequality')
    else:
        print('>>> Do not satisfy Triangle Inequality')
    
    i += 1

print(80 * "-")    
print(str(num_satisfied/num_samples))

-------------------- triangle 1 --------------------
edge 1: Census Tract 021815 -- Census Tract 011502 
weight: 442.71
edge 2: Census Tract 482701 -- Census Tract 408211 
weight: 1050.01
edge 3: Census Tract 532304 -- Census Tract 544002 
weight: 1167.51
>>> Satisfy Triangle Inequality
-------------------- triangle 2 --------------------
edge 1: Census Tract 203300 -- Census Tract 086802 
weight: 2270.81
edge 2: Census Tract 209402 -- Census Tract 186202 
weight: 1149.57
edge 3: Census Tract 228210 -- Census Tract 212203 
weight: 1417.43
>>> Satisfy Triangle Inequality
-------------------- triangle 3 --------------------
edge 1: Census Tract 406200 -- Census Tract 403600 
weight: 384.24
edge 2: Census Tract 220100 -- Census Tract 267502 
weight: 1523.08
edge 3: Census Tract 217002 -- Census Tract 231900 
weight: 1214.53
>>> Satisfy Triangle Inequality
-------------------- triangle 4 --------------------
edge 1: Census Tract 601700 -- Census Tract 011708 
weight: 2414.29
edge 2: Census

edge 2: Census Tract 231720 -- Census Tract 573300 
weight: 1616.53
edge 3: Census Tract 212305 -- Census Tract 195202 
weight: 1193.79
>>> Satisfy Triangle Inequality
-------------------- triangle 73 --------------------
edge 1: Census Tract 106646 -- Census Tract 195100 
weight: 1788.95
edge 2: Census Tract 106642 -- Census Tract 292000 
weight: 2724.49
edge 3: Census Tract 602801 -- Census Tract 575801 
weight: 1339.62
>>> Satisfy Triangle Inequality
-------------------- triangle 74 --------------------
edge 1: Census Tract 267300 -- Census Tract 800203 
weight: 1889.76
edge 2: Census Tract 702802 -- Census Tract 262200 
weight: 709.13
edge 3: Census Tract 540600 -- Census Tract 553400 
weight: 376.22
>>> Do not satisfy Triangle Inequality
-------------------- triangle 75 --------------------
edge 1: Census Tract 543501 -- Census Tract 670403 
weight: 1388.65
edge 2: Census Tract 123700 -- Census Tract 980021 
weight: 969.57
edge 3: Census Tract 240600 -- Census Tract 231600 
weight

weight: 1231.41
edge 3: Census Tract 206010 -- Census Tract 408302 
weight: 1659.2
>>> Satisfy Triangle Inequality
-------------------- triangle 138 --------------------
edge 1: Census Tract 113237 -- Census Tract 137402 
weight: 1187.36
edge 2: Census Tract 110007 -- Census Tract 432802 
weight: 1750.49
edge 3: Census Tract 207101 -- Census Tract 296902 
weight: 1862.5
>>> Satisfy Triangle Inequality
-------------------- triangle 139 --------------------
edge 1: Census Tract 209810 -- Census Tract 572301 
weight: 1785.45
edge 2: Census Tract 602200 -- Census Tract 400900 
weight: 3178.5
edge 3: Census Tract 534102 -- Census Tract 239201 
weight: 1550.2
>>> Satisfy Triangle Inequality
-------------------- triangle 140 --------------------
edge 1: Census Tract 407301 -- Census Tract 406702 
weight: 509.89
edge 2: Census Tract 221900 -- Census Tract 267800 
weight: 1470.16
edge 3: Census Tract 550400 -- Census Tract 109100 
weight: 2964.23
>>> Do not satisfy Triangle Inequality
---------

edge 3: Census Tract 099906 -- Census Tract 570001 
weight: 905.18
>>> Satisfy Triangle Inequality
-------------------- triangle 203 --------------------
edge 1: Census Tract 572900 -- Census Tract 240600 
weight: 1607.13
edge 2: Census Tract 572301 -- Census Tract 292000 
weight: 632.29
edge 3: Census Tract 702502 -- Census Tract 269602 
weight: 993.41
>>> Satisfy Triangle Inequality
-------------------- triangle 204 --------------------
edge 1: Census Tract 551800 -- Census Tract 087101 
weight: 1049.19
edge 2: Census Tract 122200 -- Census Tract 194200 
weight: 2057.15
edge 3: Census Tract 702502 -- Census Tract 213202 
weight: 1115.97
>>> Satisfy Triangle Inequality
-------------------- triangle 205 --------------------
edge 1: Census Tract 502402 -- Census Tract 501900 
weight: 709.06
edge 2: Census Tract 482522 -- Census Tract 432500 
weight: 996.15
edge 3: Census Tract 110301 -- Census Tract 278102 
weight: 2464.35
>>> Do not satisfy Triangle Inequality
-------------------- tria

-------------------- triangle 268 --------------------
edge 1: Census Tract 504002 -- Census Tract 502200 
weight: 1145.36
edge 2: Census Tract 211120 -- Census Tract 241001 
weight: 1522.24
edge 3: Census Tract 575401 -- Census Tract 294701 
weight: 506.81
>>> Satisfy Triangle Inequality
-------------------- triangle 269 --------------------
edge 1: Census Tract 226002 -- Census Tract 501504 
weight: 1840.08
edge 2: Census Tract 274100 -- Census Tract 214901 
weight: 1865.54
edge 3: Census Tract 554521 -- Census Tract 577501 
weight: 953.6
>>> Satisfy Triangle Inequality
-------------------- triangle 270 --------------------
edge 1: Census Tract 291300 -- Census Tract 551600 
weight: 1181.36
edge 2: Census Tract 500900 -- Census Tract 087102 
weight: 1391.78
edge 3: Census Tract 191710 -- Census Tract 503902 
weight: 2191.25
>>> Satisfy Triangle Inequality
-------------------- triangle 271 --------------------
edge 1: Census Tract 571800 -- Census Tract 482303 
weight: 1663.02
edge 2:

...

edge 2: Census Tract 218400 -- Census Tract 502602 
weight: 1944.2
edge 3: Census Tract 238100 -- Census Tract 600601 
weight: 384.64
>>> Satisfy Triangle Inequality
-------------------- triangle 725 --------------------
edge 1: Census Tract 533202 -- Census Tract 502700 
weight: 1499.29
edge 2: Census Tract 574400 -- Census Tract 572301 
weight: 589.47
edge 3: Census Tract 194401 -- Census Tract 190301 
weight: 1084.13
>>> Satisfy Triangle Inequality
-------------------- triangle 726 --------------------
edge 1: Census Tract 143200 -- Census Tract 211410 
weight: 1014.81
edge 2: Census Tract 228310 -- Census Tract 192001 
weight: 2279.87
edge 3: Census Tract 191710 -- Census Tract 503902 
weight: 2191.25
>>> Satisfy Triangle Inequality
-------------------- triangle 727 --------------------
edge 1: Census Tract 555211 -- Census Tract 011300 
weight: 1290.64
edge 2: Census Tract 086801 -- Census Tract 226700 
weight: 2374.54
edge 3: Census Tract 191500 -- Census Tract 191204 
weight: 10

weight: 1497.95
edge 3: Census Tract 530602 -- Census Tract 203720 
weight: 533.05
>>> Satisfy Triangle Inequality
-------------------- triangle 790 --------------------
edge 1: Census Tract 404803 -- Census Tract 551300 
weight: 1496.6
edge 2: Census Tract 576901 -- Census Tract 553000 
weight: 1260.02
edge 3: Census Tract 482521 -- Census Tract 482101 
weight: 589.07
>>> Satisfy Triangle Inequality
-------------------- triangle 791 --------------------
edge 1: Census Tract 433304 -- Census Tract 480012 
weight: 990.87
edge 2: Census Tract 297602 -- Census Tract 239601 
weight: 1653.48
edge 3: Census Tract 555104 -- Census Tract 572301 
weight: 1111.9
>>> Satisfy Triangle Inequality
-------------------- triangle 792 --------------------
edge 1: Census Tract 536102 -- Census Tract 214000 
weight: 2451.19
edge 2: Census Tract 541700 -- Census Tract 403721 
weight: 2379.32
edge 3: Census Tract 221303 -- Census Tract 542106 
weight: 1796.91
>>> Satisfy Triangle Inequality
----------------

edge 3: Census Tract 185100 -- Census Tract 302301 
weight: 1094.94
>>> Satisfy Triangle Inequality
-------------------- triangle 855 --------------------
edge 1: Census Tract 930401 -- Census Tract 462700 
weight: 2208.14
edge 2: Census Tract 463602 -- Census Tract 123206 
weight: 2086.77
edge 3: Census Tract 212900 -- Census Tract 601001 
weight: 1709.45
>>> Satisfy Triangle Inequality
-------------------- triangle 856 --------------------
edge 1: Census Tract 206010 -- Census Tract 621326 
weight: 3321.4
edge 2: Census Tract 604001 -- Census Tract 403305 
weight: 2874.44
edge 3: Census Tract 204700 -- Census Tract 407802 
weight: 1585.65
>>> Satisfy Triangle Inequality
-------------------- triangle 857 --------------------
edge 1: Census Tract 226002 -- Census Tract 292000 
weight: 1730.17
edge 2: Census Tract 269000 -- Census Tract 703002 
weight: 1092.3
edge 3: Census Tract 320000 -- Census Tract 702202 
weight: 2165.92
>>> Satisfy Triangle Inequality
-------------------- triangle

-------------------- triangle 920 --------------------
edge 1: Census Tract 106642 -- Census Tract 292000 
weight: 2724.49
edge 2: Census Tract 232800 -- Census Tract 531102 
weight: 1215.92
edge 3: Census Tract 241300 -- Census Tract 534501 
weight: 1211.63
>>> Do not satisfy Triangle Inequality
-------------------- triangle 921 --------------------
edge 1: Census Tract 501200 -- Census Tract 293202 
weight: 1721.11
edge 2: Census Tract 554522 -- Census Tract 433101 
weight: 1511.57
edge 3: Census Tract 011502 -- Census Tract 110301 
weight: 660.36
>>> Satisfy Triangle Inequality
-------------------- triangle 922 --------------------
edge 1: Census Tract 231100 -- Census Tract 294900 
weight: 1188.31
edge 2: Census Tract 134103 -- Census Tract 134305 
weight: 334.01
edge 3: Census Tract 127102 -- Census Tract 191902 
weight: 1703.64
>>> Do not satisfy Triangle Inequality
-------------------- triangle 923 --------------------
edge 1: Census Tract 541700 -- Census Tract 403721 
weight: 

edge 1: Census Tract 980013 -- Census Tract 088105 
weight: 2612.53
edge 2: Census Tract 551900 -- Census Tract 503103 
weight: 878.82
edge 3: Census Tract 106510 -- Census Tract 189905 
weight: 1737.66
>>> Satisfy Triangle Inequality
-------------------- triangle 986 --------------------
edge 1: Census Tract 271702 -- Census Tract 536000 
weight: 2648.84
edge 2: Census Tract 554522 -- Census Tract 554512 
weight: 516.75
edge 3: Census Tract 183221 -- Census Tract 531201 
weight: 1258.14
>>> Do not satisfy Triangle Inequality
-------------------- triangle 987 --------------------
edge 1: Census Tract 621201 -- Census Tract 194401 
weight: 2777.65
edge 2: Census Tract 237600 -- Census Tract 201504 
weight: 1799.7
edge 3: Census Tract 310703 -- Census Tract 702300 
weight: 2132.52
>>> Satisfy Triangle Inequality
-------------------- triangle 988 --------------------
edge 1: Census Tract 408402 -- Census Tract 222600 
weight: 1771.43
edge 2: Census Tract 432101 -- Census Tract 406500 
wei

Now, we want to find an approximation solution for the traveling salesman problem (TSP) on G. Apply the 1-approximate algorithm described in the class. Inspect the sequence of street addresses visited on the map and see if the results are intuitive.

### Question 9:
Find an upper bound on the empirical performance of the approximate algorithm:

$\rho = \frac{Approximate TSP Cost}{Optimal TSP Cost}$

### Question 10: